In [1]:
 %pip install pdf2image pytesseract
 %pip install pandas 
 %pip install pdfplumber

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# %pip install pdf2image pytesseract
# %pip install pandas

from pdf2image import convert_from_path
import pytesseract

pytesseract.pytesseract.tesseract_cmd = "C:\\Program Files\\Tesseract-OCR\\tesseract.exe" 

def ocr_extract_text(pdf_path):
    # Convert PDF pages to images
    pages = convert_from_path(pdf_path, dpi=300, poppler_path= "C:\\Users\\chsat\\Documents\\poppler-24.08.0\\Library\\bin")
    # print(pages)
    extracted_pages_text = []
    
    for i, page_img in enumerate(pages[:3]):
        text = pytesseract.image_to_string(page_img)
        extracted_pages_text.append(text)
        
        # Optional: Save each page image for verification
        page_img.save(f"page_{i+1}.png", "PNG")
    
    return extracted_pages_text




In [ ]:
pdf_path = "D:\\Projects\\nspc-voters-project\\input\Anakapalli\\Narsipatnam\\booth_79.pdf"  # Update with your file path
ocr_pages_text = ocr_extract_text(pdf_path)


<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:1: SyntaxWarning: invalid escape sequence '\A'
C:\Users\chsat\AppData\Local\Temp\ipykernel_18052\2699081286.py:1: SyntaxWarning: invalid escape sequence '\A'
  pdf_path = "D:\\Projects\\nspc-voters-project\\input\Anakapalli\\Narsipatnam\\booth_79.pdf"  # Update with your file path


In [ ]:
ocr_pages_text

["Assembly Constituency No and Name: 61-JUBILEE HILLS\n\nSection No and Name 1-Aha Hotel Lane\n\n[\n\nName : Sourbhi Gupta\n\nHusbands Name: Mahaveer Sharan Gupta\nHouse Number : 8-1-208 Amberley heights\nAge : 34 Gender : Female\n\nWKH3919800\n\nPhoto\n\nAvailable\n\nWKH3135845\n\n[a\n\nName : Mukesh Kumar Shrivastava\nFathers Name: Sudhir Kumar Shrivastava\nHouse Number : 8-1-284, OU/148/B/A/1\nAge : 3 Gender : Male\n\nPhoto\n\nAvailable\n\nName :K.Kondamma\nHusbands Name: K Kondaiah\nHouse Number : 8-1-284/39/3\nAge : 57 Gender : Female\n\nWKH3465077\n\nPhoto\n\nAvailable\n\nName : PAVANI KARNE\n\nHusbands Name: PANDARINATH KARNE\nHouse Number : 8-1-284/A/47 & 48 Flat\n302.\n\nAge : 35 Gender : Female\n\nWKH3881570\n\nPhoto\n\nAvailable\n\nName : Jyothi Kiran ET!\n\nMothers Name: Krishna Kumari ETI\nHouse Number : 8-1-\n284/A/55T59,62T66/102\n\nAge : 43 Gender + Male\n\nWKH3960762\n\nPhoto\n\nAvailable\n\nWKH2697805\n\nName : Vampati Kalyan Kumar\nFathers Name: Vempati Bhaskar Rao\n

In [ ]:
import os

def save_ocr_output(ocr_text_list, output_dir="ocr_text_output"):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    for i, page_text in enumerate(ocr_text_list, start=1):
        with open(os.path.join(output_dir, f"page_{i}.txt"), "w", encoding="utf-8") as f:
            f.write(page_text)
    
    print(f"Saved {len(ocr_text_list)} pages of OCR output in folder: {output_dir}")




In [ ]:
# Example usage:
save_ocr_output(ocr_pages_text)  # where ocr_pages_text is the list of extracted texts

Saved 33 pages of OCR output in folder: ocr_text_output


In [ ]:
import re
import pandas as pd

# Load text
output_dir="ocr_text_output"
# for i, page_text in enumerate(ocr_text_list, start=1):
#         with open(os.path.join(output_dir, f"page_{i}.txt"), "w", encoding="utf-8") as f:
#             f.write(page_text)
for x in os.listdir(output_dir):
    if x.endswith(".txt"):
        # Prints only text file present in My Folder
        print(x)
        with open(os.path.join(output_dir, x), "r", encoding="utf-8") as f:
            text = f.read()

        # Phase 1: Split into blocks by 'Name' occurrences
        blocks = re.split(r"\n\s*(?:Name\s*[:=+\s]+)", text, flags=re.IGNORECASE)[1:]  # skip the first split part

        # Extract all voter codes beforehand
        codes = re.findall(r"\b([A-Z0-9]{10})\b", text)

        data = []

        for block in blocks:
            block = block.strip()

            # Phase 2: extract fields from each block
            name = block.split('\n')[0].strip()  # first line after "Name" is the name
            house_match = re.search(r"House Number\s*[:=]\s*(.+?)(?:\n|$)", block, re.IGNORECASE)
            house = house_match.group(1).replace("\n", " ").strip() if house_match else ""

            # Relative name: try all options
            relative_match = re.search(
                r"(Husbands Name|Fathers Name|Mothers Name|Others)[:=]\s*(.+?)(?:\n|$)", block, re.IGNORECASE)
            relative = relative_match.group(2).strip() if relative_match else ""

            age_match = re.search(r"Age\s*[:=+\s]*?(\d+)", block, re.IGNORECASE)
            age = age_match.group(1) if age_match else ""

            gender_match = re.search(r"Gender\s*[:=+\s]*?(Male|Female)", block, re.IGNORECASE)
            gender = gender_match.group(1) if gender_match else ""

            # Find the nearest voter code after this block in the full text
            block_pos = text.find(block)
            nearest_code = None
            for code in codes:
                code_pos = text.find(code, block_pos)
                if code_pos > block_pos:
                    nearest_code = code
                    break

            data.append({
                "Name": name,
                "House Number": house,
                "Relative Name": relative,
                "Age": age,
                "Gender": gender,
                "VoterID": nearest_code if nearest_code else ""
            })

        # Save to CSV
        df = pd.DataFrame(data)
        append_write = 'w'
        header_flag = True
        csv_file = "parsed_voters_robust.csv"
        if os.path.exists(csv_file):
            append_write = 'a' # append if already exists
            header_flag = False
      

        df.to_csv(csv_file, index=False, header = header_flag, mode = append_write )
        print(f"Extracted {len(df)} records.")

# with open("ocr_text_output\\page_1.txt", "r", encoding="utf-8") as f:
#     text = f.read()

# # Phase 1: Split into blocks by 'Name' occurrences
# blocks = re.split(r"\n\s*(?:Name\s*[:=+\s]+)", text, flags=re.IGNORECASE)[1:]  # skip the first split part

# # Extract all voter codes beforehand
# codes = re.findall(r"\b([A-Z0-9]{10})\b", text)

# data = []

# for block in blocks:
#     block = block.strip()

#     # Phase 2: extract fields from each block
#     name = block.split('\n')[0].strip()  # first line after "Name" is the name
#     house_match = re.search(r"House Number\s*[:=]\s*(.+?)(?:\n|$)", block, re.IGNORECASE)
#     house = house_match.group(1).replace("\n", " ").strip() if house_match else ""

#     # Relative name: try all options
#     relative_match = re.search(
#         r"(Husbands Name|Fathers Name|Mothers Name|Others)[:=]\s*(.+?)(?:\n|$)", block, re.IGNORECASE)
#     relative = relative_match.group(2).strip() if relative_match else ""

#     age_match = re.search(r"Age\s*[:=+\s]*?(\d+)", block, re.IGNORECASE)
#     age = age_match.group(1) if age_match else ""

#     gender_match = re.search(r"Gender\s*[:=+\s]*?(Male|Female)", block, re.IGNORECASE)
#     gender = gender_match.group(1) if gender_match else ""

#     # Find the nearest voter code after this block in the full text
#     block_pos = text.find(block)
#     nearest_code = None
#     for code in codes:
#         code_pos = text.find(code, block_pos)
#         if code_pos > block_pos:
#             nearest_code = code
#             break

#     data.append({
#         "Name": name,
#         "House Number": house,
#         "Relative Name": relative,
#         "Age": age,
#         "Gender": gender,
#         "VoterID": nearest_code if nearest_code else ""
#     })

# # Save to CSV
# df = pd.DataFrame(data)
# df.to_csv("parsed_voters_robust.csv", index=False)
# print(f"Extracted {len(df)} records.")



page_1.txt
Extracted 30 records.
page_10.txt
Extracted 18 records.
page_11.txt
Extracted 30 records.
page_12.txt
Extracted 30 records.
page_13.txt
Extracted 30 records.
page_14.txt
Extracted 29 records.
page_15.txt
Extracted 30 records.
page_16.txt
Extracted 30 records.
page_17.txt
Extracted 30 records.
page_18.txt
Extracted 10 records.
page_19.txt
Extracted 30 records.
page_2.txt
Extracted 30 records.
page_20.txt
Extracted 30 records.
page_21.txt
Extracted 30 records.
page_22.txt
Extracted 26 records.
page_23.txt
Extracted 20 records.
page_24.txt
Extracted 30 records.
page_25.txt
Extracted 30 records.
page_26.txt
Extracted 30 records.
page_27.txt
Extracted 30 records.
page_28.txt
Extracted 30 records.
page_29.txt
Extracted 30 records.
page_3.txt
Extracted 30 records.
page_30.txt
Extracted 3 records.
page_31.txt
Extracted 9 records.
page_32.txt
Extracted 30 records.
page_33.txt
Extracted 25 records.
page_4.txt
Extracted 30 records.
page_5.txt
Extracted 30 records.
page_6.txt
Extracted 

In [ ]:
from pdf2image import convert_from_path

images = convert_from_path('D:\\Projects\\Github\\NSPC_DATA_APP\\Backend\\booth_79.pdf', dpi=300, poppler_path= "C:\\Users\\chsat\\Documents\\poppler-24.08.0\\Library\\bin")
for i, image in enumerate(images):
    image.save(f'page_{i + 1}.jpg', 'JPEG')

In [ ]:
import cv2

# Read the image
image = cv2.imread('page_1.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply adaptive thresholding
threshold_img = cv2.adaptiveThreshold(
    gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
)

# Optionally, remove noise
processed_img = cv2.medianBlur(threshold_img, 3)


In [ ]:
# Find contours
contours, _ = cv2.findContours(
    processed_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
)

# Draw bounding boxes around contours
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Draw rectangle


In [ ]:
sub_images = []
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    sub_image = image[y:y+h, x:x+w]  # Crop sub-image
    sub_images.append(sub_image)
    cv2.imwrite(f"sub_image_{x}_{y}.jpg", sub_image)  # Save sub-images


In [ ]:
min_width, min_height = 50, 20  # Adjust as needed

filtered_contours = [
    c for c in contours if cv2.boundingRect(c)[2] > min_width and cv2.boundingRect(c)[3] > min_height
]

# Process filtered contours only
for contour in filtered_contours:
    x, y, w, h = cv2.boundingRect(contour)
    sub_image = image[y:y+h, x:x+w]
    cv2.imwrite(f"filtered_sub_image_{x}_{y}.jpg", sub_image)


In [ ]:
import pytesseract
from pytesseract import Output
pytesseract.pytesseract.tesseract_cmd = "C:\\Program Files\\Tesseract-OCR\\tesseract.exe" 
box_data = pytesseract.image_to_data(sub_image, output_type=Output.DICT)

for i in range(len(box_data['text'])):
    if box_data['text'][i].strip():  # Ignore empty text
        print(f"Text: {box_data['text'][i]}")
        print(f"Box: ({box_data['left'][i]}, {box_data['top'][i]}, {box_data['width'][i]}, {box_data['height'][i]})")


Text: Assembly
Box: (115, 35, 168, 37)
Text: Constituency
Box: (294, 35, 217, 37)
Text: No
Box: (523, 35, 45, 29)
Text: and
Box: (579, 35, 60, 29)
Text: Name
Box: (653, 35, 96, 29)
Text: :
Box: (763, 43, 5, 21)
Text: 34-Narsipatnam
Box: (782, 35, 263, 37)
Text: Section
Box: (117, 123, 107, 25)
Text: No
Box: (237, 123, 38, 25)
Text: and
Box: (286, 123, 52, 25)
Text: Name
Box: (351, 123, 82, 25)
Text: :
Box: (446, 129, 4, 19)
Text: 1-B.C
Box: (466, 123, 82, 25)
Text: COLONY
Box: (560, 123, 133, 25)
Text: -
Box: (704, 137, 11, 3)
Text: 1
Box: (728, 123, 10, 25)
Text: AEX1323997
Box: (644, 225, 192, 25)
Text: Name
Box: (115, 263, 90, 23)
Text: :
Box: (199, 258, 17, 38)
Text: LOVA
Box: (225, 263, 75, 23)
Text: BODAPALLI
Box: (309, 263, 157, 23)
Text: Husband's
Box: (115, 297, 136, 23)
Text: Name
Box: (260, 297, 91, 23)
Text: :
Box: (348, 293, 16, 37)
Text: VARAHALA
Box: (369, 297, 149, 23)
Text: BABU
Box: (526, 297, 76, 23)
Text: BODAPALLI
Box: (115, 331, 157, 23)
Text: House
Box: (115, 365

In [1]:
import pdfplumber
import re
import pandas as pd

def extract_voter_data(pdf_path):
    data = []
    current_section = ""
    
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            
            # Extract section name
            section_match = re.search(r"Section No and Name : (.+?)(\n|$)", text)
            if section_match:
                current_section = section_match.group(1).strip()
            
            # Split text into voter entries
            entries = re.split(r'\|\s*\d+\s*\|\s*[A-Z0-9]+\s*\|', text)
            for entry in entries[1:]:  # Skip the first empty split
                entry = entry.strip()
                if not entry:
                    continue
                
                # Extract fields using regex
                voter_id = re.search(r'([A-Z0-9]+)\s*\n', entry)
                voter_id = voter_id.group(1) if voter_id else None
                
                name = re.search(r'Name : (.+?)\n', entry)
                name = name.group(1).strip() if name else None
                
                relation_field = re.search(
                    r'(Mother\'s Name|Father\'s Name|Husband\'s Name) : (.+?)\n', 
                    entry
                )
                relation_type = relation_field.group(1).replace("'s Name", "") if relation_field else None
                relation = relation_field.group(2).strip() if relation_field else None
                
                house = re.search(r'House Number : (.+?)\n', entry)
                house = house.group(1).strip() if house else None
                
                age = re.search(r'Age : (\d+)', entry)
                age = age.group(1) if age else None
                
                gender = re.search(r'Gender : (\w+)(\n|$)', entry)
                gender = gender.group(1).strip() if gender else None
                
                if all([voter_id, name, relation_type, relation, house, age, gender]):
                    data.append({
                        'Voter_id': voter_id,
                        'Name': name,
                        'Relation': relation,
                        'Relation_Type': relation_type,
                        'Address': f"{current_section}, House {house}",
                        'Age': int(age),
                        'Gender': gender
                    })
    
    return pd.DataFrame(data)

# Usage
df = extract_voter_data("D:\\Projects\\nspc-voters-project\\input\\Anakapalli\\Narsipatnam\\booth_79.pdf")

# Save to CSV
df.to_csv("voter_data.csv", index=False)

# Display sample
print(f"Total records extracted: {len(df)}")

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Total records extracted: 0
